## Deep Learning for Mortality Prediction (DLMP)

### Goals of this analysis
 - assess performance of the deep learning model given US state data, HMD country data, and both datasets 
 - compare performance to baselines (Lee-Carter)

### Possible future questions worth asking: 
 - would it also improve relative to coherant LC extensions? 
 - could do a more gradual increase in sample size (compare performance adding a tenth of the states at a time) 
 - county level estimates (do this next)

### Import packages 

In [117]:
import tensorflow as tf
import csv
import numpy as np
import pandas as pd
import os as os
import matplotlib.pyplot as plt
tfkl = tf.keras.layers

### Prepare data

In [4]:
# loading in USMDB data
data = []
ages = []
states = []
genders = []

with open("../data/usmdb/usmdb.csv", "r") as file:
    reader = csv.reader(file,delimiter=',')
    for row_index, row in enumerate(reader):
        if row_index == 0:
            print(row)
        if row_index >= 1:
            state, gender, year, age, rate = row
            year = int(year)
            try:
                age = int(age)
            except:
                age = -1
            if state not in states:
                states.append(state)
            state = states.index(state)
            if gender not in genders:
                genders.append(gender)
            gender = genders.index(gender)
            try:
                rate = float(rate)
            except:
                rate = -1
            if rate > 1:
                rate = 1
            # get rid of years, ages, not in health data and other cleaning
            if age != -1 and rate != -1 and age <= 99:
                data.append([state, gender, year, age, rate])

state_data = np.array(data)

['PopName', 'Sex', 'Year', 'Age', 'mx']


In [36]:
# loading in HMD data
data = []
ages = []
countries = []
genders = []

with open("../data/Mx_1x1/hmd.csv", "r") as file:
    reader = csv.reader(file,delimiter=",")
    for row_index, row in enumerate(reader):
        if row_index == 0:
            print(row)
        if row_index >= 1:
            year, age, country, gender, rate = row
            year = int(year)
            try:
                age = int(age)
            except:
                age = -1
            if country not in countries:
                countries.append(country)
            country = countries.index(country)
            if gender not in genders:
                genders.append(gender)
            gender = genders.index(gender)
            try:
                rate = float(rate)
            except:
                rate = -1
            if rate > 1:
                rate = 1
            if age != -1 and rate != -1 and age <= 99:
                data.append([country, gender, year, age, rate])

country_data = np.array(data)

['Year', 'Age', 'Country', 'Gender', 'Mortality_rate']


In [37]:
# getting unique values for geographic location column 
country_data[:,0] = country_data[:,0] + 50

# dropping US
country_data = country_data[country_data[:,0] != 87]
countries.remove('USA')

# merge data
combined = np.vstack((state_data, country_data))

### Train deep learning model

In [ ]:
# function to create and prepare training and test sets 
def create_train_set(data, start_year, end_year) {
    training_index = np.logical_and(data[:, 2] >= 1959, data[:, 2] <= 2005)
    training_data = data[training_index, :]
    
    training_data = tf.convert_to_tensor(training_data)
    training_data = tf.cast(training_data, tf.float32)
    
}

create_test_set <- function(data, start_year, end_year) {

}

In [47]:

training_index = np.logical_and(combined[:, 2] >= 1959, combined[:, 2] <= 2005)
training_data = combined[training_index, :]
print(training_data.shape)

test_index = np.logical_and(combined[:, 2] > 2005, combined[:, 2] <= 2015)
test_data = combined[test_index, :]
print(test_data.shape)

final_test_index = np.logical_and(combined[:, 2] > 2015, combined[:, 2] <= 2019)
final_test = combined[final_test_index, :]
print(final_test.shape)

(801394, 5)
(172000, 5)
(66400, 5)


In [49]:
# prepare training and test data 
training_data = tf.convert_to_tensor(training_data)
test_data = tf.convert_to_tensor(test_data)
final_test = tf.convert_to_tensor(final_test)

training_data = tf.cast(training_data, tf.float32)
test_data = tf.cast(test_data, tf.float32)
final_test = tf.cast(final_test, tf.float32)

num_train = training_data.shape[0]
num_test = test_data.shape[0]
num_final = final_test.shape[0]

In [74]:
# define function to fetch and process data entries from training or test data 
def get_data(index, mode):
    if mode == "train":
        # randomly selects index from training data between 0 and num_train
        rand_index = tf.random.uniform([],minval=0, maxval=num_train, dtype=tf.int32) 
        entry = training_data[rand_index, :]
    elif mode == "not_random":
        # selects specified index from test data 
        entry = test_data[index, :]
    else: 
        # for any other value of mode, randomly selects index from test
        rand_index = tf.random.uniform([],minval=0, maxval=num_test, dtype=tf.int32)
        entry = test_data[rand_index, :]
    geography, gender, year, age, rate = entry[0], entry[1], entry[2], entry[3], entry[4]
    year = (year - 1998)/21
    age = tf.cast(age, tf.int32)
    geography = tf.cast(geography, tf.int32)
    gender = tf.cast(gender, tf.int32)
    year = tf.reshape(year, [1])
    age = tf.reshape(age, [1])
    geography = tf.reshape(geography, [1])
    gender = tf.reshape(gender, [1])
    rate = tf.reshape(rate, [1])
    return (year, age, geography, gender), rate

In [81]:
# use get_data function to set up training and test tensorflow datasets 
dataset_train = tf.data.Dataset.from_tensor_slices(np.arange(10000))
dataset_train = dataset_train.repeat()
dataset_train = dataset_train.map(lambda x: get_data(x, mode="train"), num_parallel_calls=4)
dataset_train = dataset_train.batch(256)
dataset_train = dataset_train.prefetch(buffer_size=512)

dataset_test = tf.data.Dataset.from_tensor_slices(np.arange(10000))
dataset_test = dataset_test.repeat()
dataset_test = dataset_test.map(lambda x: get_data(x, mode="test"), num_parallel_calls=4)
dataset_test = dataset_test.batch(256)
dataset_test = dataset_test.prefetch(buffer_size=512)

dataset_test2 = tf.data.Dataset.from_tensor_slices(np.arange(68000))
dataset_test2 = dataset_test2.map(lambda x: get_data(x, mode="not_random"), num_parallel_calls=4)
dataset_test2 = dataset_test2.batch(256)
dataset_test2 = dataset_test2.prefetch(buffer_size=512)

dataset_final = tf.data.Dataset.from_tensor_slices(np.arange(10000))
dataset_final = dataset_final.repeat()
dataset_final = dataset_final.map(lambda x: get_data(x, mode="test"), num_parallel_calls=4)
dataset_final = dataset_final.batch(256)
dataset_final = dataset_final.prefetch(buffer_size=512)

In [82]:
def create_model():
    # defining inputs 
    year = tfkl.Input(shape=(1,), dtype='float32', name='Year')
    age =  tfkl.Input(shape=(1,), dtype='int32', name='Age')
    geography = tfkl.Input(shape=(1,), dtype='int32', name='Geography')
    gender = tfkl.Input(shape=(1,), dtype='int32', name='Gender')

    # defining embedding layers 
    age_embed = tfkl.Embedding(input_dim=100, output_dim=5, input_length=1, name='Age_embed')(age)
    age_embed = tfkl.Flatten()(age_embed)

    gender_embed = tfkl.Embedding(input_dim=2, output_dim=5, input_length=1, name='Gender_embed')(gender)
    gender_embed = tfkl.Flatten()(gender_embed)

    geography_embed = tfkl.Embedding(input_dim=87, output_dim=5, input_length=1, name='Geography_embed')(geography)
    geography_embed = tfkl.Flatten()(geography_embed)

    # create feature vector that concatenates all inputs 
    x = tfkl.Concatenate()([year, age_embed, gender_embed, geography_embed])
    x1 = x

    # setting up middle layers 
    x = tfkl.Dense(128, activation='tanh')(x)
    x = tfkl.BatchNormalization()(x)
    x = tfkl.Dropout(0.05)(x)

    x = tfkl.Dense(128, activation='tanh')(x)
    x = tfkl.BatchNormalization()(x)
    x = tfkl.Dropout(0.05)(x)

    x = tfkl.Dense(128, activation='tanh')(x)
    x = tfkl.BatchNormalization()(x)
    x = tfkl.Dropout(0.05)(x)

    x = tfkl.Dense(128, activation='tanh')(x)
    x = tfkl.BatchNormalization()(x)
    x = tfkl.Dropout(0.05)(x)

    # setting up output layer 
    x = tfkl.Concatenate()([x1, x])
    x = tfkl.Dense(128, activation='tanh')(x)
    x = tfkl.BatchNormalization()(x)
    x = tfkl.Dropout(0.05)(x)
    x = tfkl.Dense(1, activation='sigmoid', name='final')(x)

    # creating the model 
    model = tf.keras.Model(inputs=[year, age, geography, gender], outputs=[x])

    # compiling the model
    model.compile(loss='mse', optimizer='adam')

    return model

In [107]:
def run_deep_model(dataset_train, dataset_test):
    
    model = create_model()

    callbacks = [tf.keras.callbacks.ReduceLROnPlateau(monitor="val_loss", factor=0.25, patience=3, verbose=0, mode="auto", 
                                                    min_delta=1e-8, cooldown=0, min_lr=0.0)]
    history = model.fit(dataset_train, steps_per_epoch=1000, validation_data=dataset_test, validation_steps=500, 
                        epochs=30, verbose=2, callbacks=callbacks)

    loss_info = {
        'train_mse': history.history['loss'][-1],
        'val_mse': history.history['val_loss'][-1]
    }

    tf.keras.backend.clear_session()

    return model, loss_info


In [108]:
model, loss_info = run_deep_model(dataset_train, dataset_test)

Epoch 1/30
1000/1000 - 11s - loss: 0.0143 - val_loss: 0.0022 - lr: 0.0010 - 11s/epoch - 11ms/step
Epoch 2/30
1000/1000 - 8s - loss: 0.0011 - val_loss: 6.9027e-04 - lr: 0.0010 - 8s/epoch - 8ms/step
Epoch 3/30
1000/1000 - 8s - loss: 7.6880e-04 - val_loss: 3.1531e-04 - lr: 0.0010 - 8s/epoch - 8ms/step
Epoch 4/30
1000/1000 - 8s - loss: 5.8665e-04 - val_loss: 5.0100e-04 - lr: 0.0010 - 8s/epoch - 8ms/step
Epoch 5/30
1000/1000 - 8s - loss: 4.8229e-04 - val_loss: 2.0311e-04 - lr: 0.0010 - 8s/epoch - 8ms/step
Epoch 6/30
1000/1000 - 8s - loss: 4.4073e-04 - val_loss: 2.0761e-04 - lr: 0.0010 - 8s/epoch - 8ms/step
Epoch 7/30
1000/1000 - 9s - loss: 4.2549e-04 - val_loss: 1.9609e-04 - lr: 0.0010 - 9s/epoch - 9ms/step
Epoch 8/30
1000/1000 - 8s - loss: 3.6890e-04 - val_loss: 2.6019e-04 - lr: 0.0010 - 8s/epoch - 8ms/step
Epoch 9/30
1000/1000 - 9s - loss: 3.5650e-04 - val_loss: 1.7954e-04 - lr: 0.0010 - 9s/epoch - 9ms/step
Epoch 10/30
1000/1000 - 9s - loss: 3.4732e-04 - val_loss: 2.3370e-04 - lr: 0.0010 

In [85]:
print(loss_info)

0.00016744744789320976


### MSE for states only from combined model

In [100]:
# prep state test set 
state_test_index = np.logical_and(state_data[:, 2] > 2005, state_data[:, 2] <= 2015)
state_test_data = state_data[state_test_index, :]
state_test_data = tf.convert_to_tensor(state_test_data)
state_test_data = tf.cast(state_test_data, tf.float32)

(100000, 5)


In [102]:
# define function to fetch and process data entries from state training or test data 
def get_state_data(index, mode):
    if mode == "train":
        # randomly selects index from training data between 0 and num_train
        rand_index = tf.random.uniform([],minval=0, maxval=num_train, dtype=tf.int32) 
        entry = training_data[rand_index, :]
    elif mode == "not_random":
        # selects specified index from test data 
        entry = state_test_data[index, :]
    else: 
        # for any other value of mode, randomly selects index from test
        rand_index = tf.random.uniform([],minval=0, maxval=num_test, dtype=tf.int32)
        entry = test_data[rand_index, :]
    geography, gender, year, age, rate = entry[0], entry[1], entry[2], entry[3], entry[4]
    year = (year - 1998)/21
    age = tf.cast(age, tf.int32)
    geography = tf.cast(geography, tf.int32)
    gender = tf.cast(gender, tf.int32)
    year = tf.reshape(year, [1])
    age = tf.reshape(age, [1])
    geography = tf.reshape(geography, [1])
    gender = tf.reshape(gender, [1])
    rate = tf.reshape(rate, [1])
    return (year, age, geography, gender), rate

In [103]:
dataset_state_test = tf.data.Dataset.from_tensor_slices(np.arange(100000))
dataset_state_test = dataset_state_test.map(lambda x: get_state_data(x, mode="not_random"), num_parallel_calls=4)
dataset_state_test = dataset_state_test.batch(256)
dataset_state_test = dataset_state_test.prefetch(buffer_size=512)

In [104]:
# generate state predictions given model 
predictions = model.predict(dataset_state_test)

391/391 [==============================] - 3s 7ms/step


In [106]:
# get the true values from the test dataset
true_values = []
for _, rate in dataset_state_test:
    true_values.extend(rate.numpy())

# convert true_values to a numpy array
true_values = np.array(true_values)

# convert predictions to a numpy array if not already
predictions = np.array(predictions)

# compute MSE using TensorFlow
mse = np.mean((true_values - predictions)**2)
print("Mean Squared Error:", mse)

Mean Squared Error: 3.246199e-05


### MSE for countries only from combined model

In [111]:
# prep country test set 
country_test_index = np.logical_and(country_data[:, 2] > 2005, country_data[:, 2] <= 2015)
country_test_data = country_data[country_test_index, :]
country_test_data = tf.convert_to_tensor(country_test_data)
country_test_data = tf.cast(country_test_data, tf.float32)

(72000, 5)


In [113]:
# define function to fetch and process data entries from country training or test data 
def get_country_data(index, mode):
    if mode == "train":
        # randomly selects index from training data between 0 and num_train
        rand_index = tf.random.uniform([],minval=0, maxval=num_train, dtype=tf.int32) 
        entry = training_data[rand_index, :]
    elif mode == "not_random":
        # selects specified index from test data 
        entry = country_test_data[index, :]
    else: 
        # for any other value of mode, randomly selects index from test
        rand_index = tf.random.uniform([],minval=0, maxval=num_test, dtype=tf.int32)
        entry = test_data[rand_index, :]
    geography, gender, year, age, rate = entry[0], entry[1], entry[2], entry[3], entry[4]
    year = (year - 1998)/21
    age = tf.cast(age, tf.int32)
    geography = tf.cast(geography, tf.int32)
    gender = tf.cast(gender, tf.int32)
    year = tf.reshape(year, [1])
    age = tf.reshape(age, [1])
    geography = tf.reshape(geography, [1])
    gender = tf.reshape(gender, [1])
    rate = tf.reshape(rate, [1])
    return (year, age, geography, gender), rate

In [114]:
dataset_country_test = tf.data.Dataset.from_tensor_slices(np.arange(72000))
dataset_country_test = dataset_country_test.map(lambda x: get_country_data(x, mode="not_random"), num_parallel_calls=4)
dataset_country_test = dataset_country_test.batch(256)
dataset_country_test = dataset_country_test.prefetch(buffer_size=512)

In [115]:
predictions = model.predict(dataset_country_test)

282/282 [==============================] - 2s 5ms/step


In [116]:
# Get the true values from the test dataset
true_values = []
for _, rate in dataset_country_test:
    true_values.extend(rate.numpy())

# Convert true_values to a numpy array
true_values = np.array(true_values)

# Convert predictions to a numpy array if not already
predictions = np.array(predictions)

# Compute MSE using TensorFlow
mse = tf.reduce_mean(tf.square(true_values - predictions)).numpy()
print("Mean Squared Error:", mse)

Mean Squared Error: 0.0002767175


### Train Lee-Carter model

In [ ]:
# non-tensor train / test split (same years in training / test here as in method above)
training_index = np.logical_and(data[:, 2] >= 1959, data[:, 2] <= 2005)
training_data = data[training_index, :]
print(training_data.shape)

test_index = np.logical_and(data[:, 2] > 2005, data[:, 2] <= 2015)
test_data = data[test_index, :]

final_test_index = np.logical_and(data[:, 2] > 2015, data[:, 2] <= 2019)
final_test = data[final_test_index, :]

(470000, 5)


In [ ]:
# set up lee-carter function

def lee_carter(mx_matrix):
    """
    Run the Lee-Carter model on age-specific mortality data.
    
    Args:
        mx_matrix (numpy.ndarray): A 2D array of age-specific mortality rates. rows = age, columns = years
        
    Returns:
        tuple: A tuple containing the estimated parameters (ax, bx, kt) and the fitted mortality rates.
    """
    mx_matrix[mx_matrix <= 0] = 1e-9

    ax = np.mean(np.log(mx_matrix), axis=1)
    ax = ax.reshape(-1, 1) # reshape ax into column vector
    
    centered_mx = np.log(mx_matrix) - ax
    
    # SVD
    U, S, Vt = np.linalg.svd(centered_mx, full_matrices=False)

    # extract right and left singular vectors (bx and kt)
    bx = U[:, 0]
    kt = Vt[0, :]
    # print(kt)

    # normalize bx and kt 
    bx = bx / np.sum(bx)
    # print(np.mean(kt))
    kt = kt - np.mean(kt)
    # print(np.mean(kt))

    # estimate fitted mortality 
    fitted_mort = np.exp(ax + np.outer(bx, kt))

    return (ax, bx, kt), fitted_mort

In [ ]:
# set up function to run multiple models on all years in training data

def lee_carter_state_gender(data):

    states = np.unique(data[:, 0])
    genders = np.unique(data[:, 1])

    results = {}

    for state in states:
        for gender in genders:
            mask = (data[:, 0] == state) & (data[:, 1] == gender)
            state_gender_data = data[mask]

            # extract ages and years
            years = np.unique(state_gender_data[:, 2])
            ages = np.unique(state_gender_data[:, 3])

            m_x = np.zeros((len(ages), len(years)))

            for i, age in enumerate(ages):
                for j, year in enumerate(years):
                    mask = (state_gender_data[:, 3] == age) & (state_gender_data[:, 2] == year)
                    m_x[i,j] = state_gender_data[mask, 4]

            params, fitted_mort = lee_carter(m_x)
    
            # Store the results for the current state and gender
            results[(state, gender)] = {
                'params': params,
                'fitted_mortality': fitted_mort
            }
    
    return results
            

In [ ]:
def lee_carter_forecast(results, h, start_year, ages, drift=True):
    """
    Perform the forecasting step of the Lee-Carter method using a random walk with drift.
    
    Args:
        results (dict): A dictionary containing the estimated parameters (ax, bx, kt) for each state and gender combination.
        h (int): The number of future periods to forecast.
        start_year (int): The starting year of the forecast.
        ages (numpy.ndarray): A 1D array of ages corresponding to the rows of the mortality matrix.
        drift (bool, optional): Whether to include a drift term in the random walk. Default is True.
        
    Returns:
        numpy.ndarray: A 2D array with 5 columns representing state, gender, year, age, and forecasted mortality rate.
    """
    
    forecasts = []
    
    for state, gender in results.keys():
        ax, bx, kt = results[(state, gender)]['params']
        
        # Estimate the drift term
        if drift:
            drift_term = (kt[-1] - kt[0]) / (len(kt) - 1)
        else:
            drift_term = 0
        
        # Forecast future kt values using a random walk with drift
        kt_forecast = np.zeros(h)
        kt_forecast[0] = kt[-1]
        for i in range(1, h):
            kt_forecast[i] = kt_forecast[i-1] + drift_term + np.random.normal(0, 1)
        
        # Forecast future mortality rates
        ax_matrix = np.repeat(ax, h).reshape(-1, h)
        bx_matrix = np.repeat(bx, h).reshape(-1, h)
        kt_matrix = np.repeat(kt_forecast, len(ax)).reshape(h, -1).T
        mortality_forecast = np.exp(ax_matrix + bx_matrix * kt_matrix)

        # Create a 2D array with state, gender, year, age, and forecasted mortality rate
        for i in range(h):
            year = start_year + i
            for j, age in enumerate(ages):
                forecasts.append([state, gender, year, age, mortality_forecast[j, i]])

    # Convert forecasts to a NumPy array
    forecasts = np.array(forecasts)

    # Sort the forecasts array based on the first four columns
    sorted_indices = np.lexsort((forecasts[:, 3], forecasts[:, 2], forecasts[:, 1], forecasts[:, 0]))
    forecasts = forecasts[sorted_indices]

    
    return forecasts

In [ ]:
def calculate_mse(forecasted_rates, actual_rates):
    """
    Calculate the Mean Squared Error (MSE) between the forecasted and actual mortality rates.
    
    Args:
        forecasted_rates (numpy.ndarray): A 2D array with 5 columns representing state, gender, year, age, and forecasted mortality rate.
        actual_rates (numpy.ndarray): A 2D array with 5 columns representing state, gender, year, age, and actual mortality rate.
        
    Returns:
        float: The Mean Squared Error (MSE) between the forecasted and actual mortality rates.
    """
    
    # Extract the forecasted and actual mortality rates
    forecasted_values = forecasted_rates[:, 4]
    actual_values = actual_rates[:, 4]
    
    # Calculate the squared differences between the forecasted and actual rates
    squared_differences = (forecasted_values - actual_values) ** 2
    
    # Calculate the Mean Squared Error (MSE)
    mse = np.mean(squared_differences)
    
    return mse

In [ ]:
def run_lc_model(train_data, test_data):
    lc_output = lee_carter_state_gender(train_data)
    predictions = lee_carter_forecast(lc_output, h=10, start_year=2006, ages=range(0, 100))
    test_mse = calculate_mse(predictions, test_data)

    return np.array(test_mse)

In [ ]:
lc_results = run_lc_model(train_data=training_data, test_data=test_data)

### Generate Table 1: Training and Test MSEs
This table will document average MSEs (for states alone, countries alone, and total) over 5 training runs with each model (LC, deep learning seperate, deep learning joint)

In [ ]:
def compare_models(num_iterations):
    results = []
    for i in range(num_iterations):
        lc_results = run_lc_model(train_data=training_data, test_data=test_data)
        deep_results = run_deep_model(dataset_train=dataset_train, dataset_test=dataset_test)
        results.append((deep_results, lc_results))
        print(f"Loop {i}: deep mse {results[i][0]} & lc mse {results[i][1]}")

    return results
        

In [ ]:
comparison_results = compare_models(num_iterations=5)

/var/folders/1z/wn6shwbs4_9gcwtsrgz8v5vc0000gn/T/ipykernel_88639/1271048323.py:24: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  m_x[i,j] = state_gender_data[mask, 4]


Loop 0: deep mse 4.0398939745500684e-05 & lc mse 0.0003915247299684933
Loop 1: deep mse 6.0458645748440176e-05 & lc mse 0.00039128320488239905
Loop 2: deep mse 4.817579247173853e-05 & lc mse 0.0003891320282911292
Loop 3: deep mse 5.007610161555931e-05 & lc mse 0.0003862197077289283
Loop 4: deep mse 4.09356398449745e-05 & lc mse 0.00038688559897930323
Loop 5: deep mse 4.911636278848164e-05 & lc mse 0.0003864877662411712
Loop 6: deep mse 4.301398075767793e-05 & lc mse 0.0003892479426403891
Loop 7: deep mse 4.956285556545481e-05 & lc mse 0.00038826033560748026
Loop 8: deep mse 4.017120591015555e-05 & lc mse 0.0003908170289735953
Loop 9: deep mse 5.103383955429308e-05 & lc mse 0.00038729158184310007
Loop 10: deep mse 4.477219408727251e-05 & lc mse 0.0003893728425534582
Loop 11: deep mse 4.873637590208091e-05 & lc mse 0.00039045673061155
Loop 12: deep mse 4.5513785153161734e-05 & lc mse 0.0003895944230450417
Loop 13: deep mse 7.137803186196834e-05 & lc mse 0.00038838044018389434
Loop 14: de

In [ ]:
comp_results_np = np.array(comparison_results)